In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 200})

## Скрытые марковские модели: дискретный случай

In [4]:
from hmmlearn import hmm

In [5]:
alphabet = np.array(['A', 'C', 'G', 'T'])
alphabet_map = { s : i for i,s in enumerate(alphabet) }
state1_prob = np.array([ .45, .05, .05, .45 ])
state2_prob = np.array([ .05, .45, .45, .05 ])
A = np.array([[ .95, .05], [.05, .95]])
B = np.array([state1_prob, state2_prob])

In [6]:
def generate_string(length=100, A=A, B=B, pi=None):
    states, m = [0], len(alphabet)
    if pi is not None:
        states = [np.random.choice(2,p=pi)]
    for i in range(length-1):
        states.append( np.random.choice(2, p=A[states[-1]]) )
    states = np.array(states)
    num0, num1 = len(states)-np.sum(states), np.sum(states)
    observables = np.zeros(len(states), dtype=int)
    observables[np.where(states==0)] = np.random.choice(m, p=B[0], size=num0)
    observables[np.where(states==1)] = np.random.choice(m, p=B[1], size=num1)
    return states, ''.join(alphabet[observables])

In [7]:
X, Z = [], []
for i in range(2000):
    z, x = generate_string(length=200)
    X.append(x)
    Z.append(z)

In [8]:
print('%s\n%s' % (''.join(['%s' % i for i in Z[10]]), X[10]))

00000000000000000000000000000000011111111110000000000111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111000000000000000000000000000000
ACAATAACAATTTATAATTCAATTTTTATATAACCCGGGCCGGTAAATAAAAAGCGGCGCCCATAATGAAAATATACTATCGAAAGTCTTATTATTTATTCATAAATATACTTAAATTTTAAGATTGAAAATAAAAAAAAATATTTGGGCGCGACGCCGCCCGCCGGCCGATAATTTTATAAAATACACTTATAAAATTT


In [9]:
model = hmm.MultinomialHMM(n_components=2, n_iter=20)
model.n_features=len(alphabet)

In [10]:
data = np.array([[ alphabet_map[s] for t in X for s in t ]]).T
lengths = np.array([len(t) for t in X])
cum_lengths = np.cumsum(lengths)

In [11]:
model.fit(data, lengths=lengths)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=2,
               n_iter=20, params='ste',
               random_state=RandomState(MT19937) at 0x7FB0836EAAF0,
               startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
               verbose=False)

In [12]:
print("Матрица переходов:\n%s\n\n" % model.transmat_)
print("\n\n".join(["=== Состояние %d ===\n" % i + "\n".join([ "%s: %.4f" % (alphabet[i], x) for i,x in sorted([ [ii, xx] for ii,xx in enumerate(model.emissionprob_[i])], key = lambda x: x[0])[:10]]) for i in range(model.emissionprob_.shape[0])]))

Матрица переходов:
[[0.5802 0.4198]
 [0.3895 0.6105]]


=== Состояние 0 ===
A: 0.1046
C: 0.2831
G: 0.3348
T: 0.2774

=== Состояние 1 ===
A: 0.4065
C: 0.1962
G: 0.1505
T: 0.2468


In [13]:
iString = 60

print( "".join(['%s' % i for i in Z[iString] ]))
print( "".join(['%s' % (1-i) for i in model.decode(data[(cum_lengths[iString-1] if iString > 0 else 0):cum_lengths[iString]])[1] ]))

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
ax.plot(Z[iString]-.05)
ax.plot( model.decode(data[(cum_lengths[iString-1] if iString > 0 else 0):cum_lengths[iString]])[1] )


00011111111111111111111111111111001111111111111111111111111111111111111111111111111111111100001111111111111111111111111111000000000011111111111111111111111111111111111111111111111111111111111000000000
00111111101110111111011111111111001111111111110111111111111110011011111111111111111111111110111110111111111111111111111110001100000000111110111011111111111111111111111111011111111111111110111000000000


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fb0864a8170> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 3000x1600 with 1 Axes>

## Скрытые марковские модели: непрерывный случай

In [14]:
A = np.array([[ .95, .05], [.05, .95]])
Bmu, Bsigma = np.array([ -1., 1. ]), np.array([ 1.0, 1.5 ])

def generate_time_series(length=100, A=A, Bmu=Bmu, Bsigma=Bsigma, pi=None):
    states, m = [0], len(alphabet)
    if pi is not None:
        states = [np.random.choice(2,p=pi)]
    for i in range(length-1):
        states.append( np.random.choice(2, p=A[states[-1]]) )
    states = np.array(states)
    num0, num1 = len(states)-np.sum(states), np.sum(states)
    observables = np.zeros(len(states))
    observables[np.where(states==0)] = np.random.normal(Bmu[0], Bsigma[0], num0)
    observables[np.where(states==1)] = np.random.normal(Bmu[1], Bsigma[1], num1)
    return states, observables

In [15]:
X, Z = [], []
for i in range(500):
    z, x = generate_time_series(length=200)
    X.append(x)
    Z.append(z)

In [16]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
ax.plot(X[270], label="Данные", color='black', linewidth=1)
ax.plot( Bmu[ Z[270] ], label="Средние состояний", linewidth=2)
plt.legend()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fb0864a8170> (for post_execute):


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 3000x1600 with 1 Axes>

In [17]:
data = np.array(X).reshape(-1, 1)
lengths = np.array([len(t) for t in X])
cum_lengths = np.cumsum(lengths)

In [18]:
data.shape

(100000, 1)

In [19]:
model = hmm.GaussianHMM(n_components=2)
model.n_features=1

In [20]:
model.fit(data, lengths=lengths)

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=2, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

In [26]:
a = '😀'
print(a)

😀


In [21]:
model.covars_

array([[[2.1701]],

       [[0.9968]]])

In [22]:
print("Матрица переходов:\n%s\n\n" % model.transmat_)
print("\n\n".join(["=== Состояние %d ===\n" % i + "%3.5f +- %02.5f" % (model.means_[i], np.sqrt(model.covars_[i])) for i in range(model.n_components)]))

Матрица переходов:
[[0.9377 0.0623]
 [0.0599 0.9401]]


=== Состояние 0 ===
1.04751 +- 1.47314

=== Состояние 1 ===
-0.99872 +- 0.99839


In [ ]:
iString = 102

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
ax.plot( model.means_[ model.decode(data[(cum_lengths[iString-1] if iString > 0 else 0):cum_lengths[iString]])[1] ], label="Средние состояний модели", linewidth=2 )
ax.plot( Bmu[ Z[iString] ] + 0.3, label="Средние состояний", linewidth=2)
ax.plot(X[iString], label="Данные", color='black', linewidth=1)
plt.legend()